In [ ]:
n_features = 1000 # Number of features
n_samples = 36570 # Number of samples
n_components = 100 # Number of components
n_top_words = 10  # Number of top words

# Use tf-idf features for NMF.
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(features)

# Use tf (raw term count) features for LDA.
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(features)

joblib.dump(tfidf, 'tfidf_features.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(tf, 'tf_features.pkl')
joblib.dump(tf_vectorizer, 'tf_vectorizer.pkl')

Start here

In [7]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

In [3]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [17]:
# Load models and data structures
views_est = joblib.load('tfidf_views_GradientBoostingRegressor.pkl') 
price_est = joblib.load('tfidf_price_GradientBoostingRegressor.pkl') 
num_favorers_est = joblib.load('tfidf_num_favorers_GradientBoostingRegressor.pkl') 
lda = joblib.load('tf_lda.pkl') 
nmf = joblib.load('tfidf_nmf.pkl')
listings = joblib.load('listings.pkl')
features = joblib.load('all_features.pickle')
tf_features = joblib.load('tf_features.pkl')
tfidf_features = joblib.load('tfidf_features.pkl')
tf_vectorizer = joblib.load('tf_vectorizer.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

tf_feature_names = tf_vectorizer.get_feature_names()
prices = np.nan_to_num(listings['price'].tolist())
views = np.nan_to_num(listings['views'].tolist())
favorers = np.nan_to_num(listings['num_favorers'].tolist())
test_size = 0.1

tfidf_price_train, tfidf_price_test, y_price_train, y_price_test = train_test_split(
    tfidf_features, prices, test_size = test_size)

tfidf_views_train, tfidf_views_test, y_views_train, y_views_test = train_test_split(
    tfidf_features, views, test_size = test_size)

tfidf_num_favorers_train, tfidf_num_favorers_test, y_num_favorers_train, y_num_favorers_test = train_test_split(
    tfidf_features, favorers, test_size = test_size)

n_features = 1000 # Number of features
n_samples = 36570 # Number of samples
n_components = 100 # Number of components
n_top_words = 10  # Number of top words

In [22]:
print(mean_absolute_error(y_price_test, price_est.predict(tfidf_price_test)))
print(explained_variance_score(y_price_test, price_est.predict(tfidf_price_test)))
print(r2_score(y_price_test, price_est.predict(tfidf_price_test)))

5.355894896356402
0.9492146261844375
0.9492129759370822


In [ ]:
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

In [ ]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [5]:
#Test price regression
mae_prices = mean_absolute_error(y_price_test, price_est.predict(tfidf_price_test))
print("MAE: %.4f" % mae_prices)

MAE: 5.1498


In [6]:
#Test views regression
mae_views = mean_absolute_error(y_views_test, views_est.predict(tfidf_views_test))
print("MAE: %.4f" % mae_views)

MAE: 416.0068


In [ ]:
# Return a list of top features for a given topic group in a model
def model_topic_features(model, feature_names, n_top_words, topic_group):
    for topic_idx, topic in enumerate(model.components_):
        if (topic_idx == topic_group):
            result = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    return(result)

In [ ]:
# Return a list of the top features for a given topic group in a model that are not in the entry list of features
def top_ten_features(model_features, entry_features, num_features):
    new_features = 0
    index = 0
    new_features_list = []
    while new_features < num_features:
        if model_features[index] != entry_features[index]:
            index += 1
            new_features += 1
            new_features_list.append(model_features)
        else
            index += 1
    return new_features_list